# Actividades Web of Data Comisión A

## Integrantes:

* Del Villar, Javier
* Otrino, Facundo Damián
* Pistoya, Haydeé Soledad
* Rojas, Mariano Arturo
* Vaillard, Leandro Carlos

## Cargamos Librerías

In [1]:
import pandas as pd
import json

## Cargamos JSONs

In [2]:
json_ecartelera = pd.read_json('WW1984ecartelera.json', orient='index')
json_IMDB = pd.read_json('WW1984IMDB.json', orient='index')
json_Tomatoes = pd.read_json('WW1984Tomatoes.json', orient='index')

## Unificamos JSONs

In [3]:
df = pd.concat([json_ecartelera, json_IMDB, json_Tomatoes], axis=1)
df.columns = ['eCartelera', 'IMDB', 'RottenTomatoes']
df

,eCartelera,IMDB,RottenTomatoes
@context,http://schema.org,https://schema.org,http://schema.org
@type,Movie,Movie,Movie
@id,https://www.ecartelera.com/peliculas/wonder-wo...,NaN,NaN
name,Wonder Woman 1984,Wonder Woman 1984,Wonder Woman 1984
mainEntityOfPage,https://www.ecartelera.com/peliculas/wonder-wo...,NaN,NaN
description,&#39;Wonder Woman 1984&#39; llegará a la panta...,Diana must contend with a work colleague and b...,NaN
duration,PT2H31M,PT2H31M,NaN
image,"{'@type': 'ImageObject', 'url': 'https://img.e...",https://m.media-amazon.com/images/M/MV5BYTlhNz...,https://resizing.flixster.com/sr2SNIPoCFaMLQt-...
countryOfOrigin,EE.UU.,NaN,NaN
genre,"[Acción, Aventura, Fantasía]","[Action, Adventure, Fantasy]","[Action & Adventure, Science Fiction & Fantasy]"


## Seleccionamos y combinamos valores de cada fuente de datos

In [4]:
json_total = pd.Series(dtype='float64')

In [5]:
json_total['@context'] = df.loc['@context', 'eCartelera']

json_total['@type'] = df.loc['@type', 'eCartelera']

json_total['name'] = df.loc['name', 'IMDB']

json_total['description'] = df.loc['description', 'eCartelera']

json_total['duration'] = df.loc['duration', 'IMDB']

json_total['image'] = df.loc['image', 'IMDB']

json_total['countryOfOrigin'] = df.loc['countryOfOrigin', 'eCartelera']

json_total['genre'] = df.loc['genre', 'IMDB']

json_total['productionCompany'] = df.loc['productionCompany', 'RottenTomatoes']

json_total['director'] = df.loc['productionCompany', 'RottenTomatoes']

json_total['actor'] = df.loc['actors', 'RottenTomatoes']

json_total['aggregateRating'] = (
    float(df.loc['aggregateRating', 'eCartelera']['ratingValue'].replace(',', '.')) + 
    df.loc['aggregateRating', 'IMDB']['ratingValue'] + 
    float(df.loc['aggregateRating', 'RottenTomatoes']['ratingValue']) / 10
    ) / 3

json_total['url'] = [
    df.loc['@id', 'eCartelera'], 
    'https://www.imdb.com' + df.loc['url', 'IMDB'], 
    df.loc['url', 'RottenTomatoes']
    ]

json_total['contentRating'] = df.loc['contentRating', 'RottenTomatoes']

json_total['trailer'] = df.loc['hasPart', 'eCartelera']

json_total['datePublished'] = df.loc['releasedEvent', 'eCartelera']

json_total['keywords'] = df.loc['keywords', 'IMDB']

json_total['review'] = df.loc['review', 'RottenTomatoes']

json_total['character'] = df.loc['character', 'RottenTomatoes']

json_total['author'] = df.loc['creator', 'IMDB']

json_total

@context                                             http://schema.org
@type                                                            Movie
name                                                 Wonder Woman 1984
description          &#39;Wonder Woman 1984&#39; llegará a la panta...
duration                                                       PT2H31M
image                https://m.media-amazon.com/images/M/MV5BYTlhNz...
countryOfOrigin                                                 EE.UU.
genre                                     [Action, Adventure, Fantasy]
productionCompany    {'@type': 'Organization', 'name': 'Warner Bros...
director             {'@type': 'Organization', 'name': 'Warner Bros...
actor                [{'@type': 'Person', 'name': 'Gal Gadot', 'sam...
aggregateRating                                                    6.3
url                  [https://www.ecartelera.com/peliculas/wonder-w...
contentRating                                                    PG-13
traile

## Exportamos el JSON totalizado

In [6]:
json_total.to_json('json_total.json')